# Wk18 Lecture02 CodeAlong: UFOs

## Learning Objectives

- By the end of this CodeAlong, students will be able to:
   - Calculate time series statistics (rolling mean/std/diff/pct_change
   - Perform feature engineering for time series EDA 
   - Aggregate time series using date parts to answer stakeholder questions.

    

# 🕹️Part 1) Preparing Irregular-Interval Time Series

### Overview from Last Lecture

- 1) [ ] Convert the dates & times to a single column (if needed).
- 2) [ ] Convert the datetime column  (most likely a string) to a datetime data type.
- 3) [ ] Set the datetime column as the Series/DataFrame index
- 4) [ ] Resample the time series to the desired/correct frequency using the desired/correct aggregation method.
- 5) [ ] Impute null values (if required)


### UFO Sightings

- UFO Sightings: https://www.kaggle.com/datasets/NUFORC/ufo-sightings 

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticks
import seaborn as sns


import missingno as miss
import datetime as dt
import statsmodels.tsa.api as tsa

plt.rcParams['figure.figsize'] = [10,5]

In [10]:
ufo  = pd.read_csv("Data/ufos-kaggle/scrubbed.csv", low_memory=False)
ufo

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611
...,...,...,...,...,...,...,...,...,...,...,...
80327,9/9/2013 21:15,nashville,tn,us,light,600,10 minutes,Round from the distance/slowly changing colors...,9/30/2013,36.1658333,-86.784444
80328,9/9/2013 22:00,boise,id,us,circle,1200,20 minutes,Boise&#44 ID&#44 spherical&#44 20 min&#44 10 r...,9/30/2013,43.6136111,-116.202500
80329,9/9/2013 22:00,napa,ca,us,other,1200,hour,Napa UFO&#44,9/30/2013,38.2972222,-122.284444
80330,9/9/2013 22:20,vienna,va,us,circle,5,5 seconds,Saw a five gold lit cicular craft moving fastl...,9/30/2013,38.9011111,-77.265556


In [11]:
ufo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80332 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   datetime              80332 non-null  object 
 1   city                  80332 non-null  object 
 2   state                 74535 non-null  object 
 3   country               70662 non-null  object 
 4   shape                 78400 non-null  object 
 5   duration (seconds)    80332 non-null  object 
 6   duration (hours/min)  80332 non-null  object 
 7   comments              80317 non-null  object 
 8   date posted           80332 non-null  object 
 9   latitude              80332 non-null  object 
 10  longitude             80332 non-null  float64
dtypes: float64(1), object(10)
memory usage: 6.7+ MB


>- 1) [x] Convert the dates & times to a single column (if needed).

## Preparing the Datetime Index

### Converting Date Cols to Datetime

In [12]:
## convert datetime to datetime
ufo['datetime'] = pd.to_datetime(ufo['datetime'])

ParserError: hour must be in 0..23: 10/11/2006 24:00 present at position 388

#### Handling Errors with pd.to_datetime

- Can use the `errors` argument for pd.to_datetime:
    - "raise" (default): raise an exception when errors happen
    - 'ignore': ignores the errors and returns the original value for that row. 
        - NOT RECOMMENDED: the entire column will not be datetime.
    - 'coerce': convert any bad datetime values to null values (NaT - NotATime)

>- **Branch point: we have a choice on how we deal with the bad timestamps.**
    -  Do we coerce them, make then null values, and drop them? Potentially losing a lot of data.
    - Or do we investigate a bit more to see if we can fix the problem without losing data.
    
    
- Let's see how much data we would lose if we chose to coerce the bad values:

In [ ]:
##check nulls before coercing errors
ufo.isna().sum()

In [ ]:
## saving a copy of original datetime column as datetime-original
ufo['datetime-original'] = ufo['datetime'].copy()
ufo

In [ ]:
## Create a datetime-coerce column using pd.to_Datetime with errors = "coerce"


In [ ]:
## How many null values did we create?


In [ ]:
## What % of dates became null?


#### Acceptable # of Rows Lost, but there is another solution!

In [ ]:
## inspecting just the bad rows
bad_rows = None
bad_rows

In [ ]:
## let's try to convert the bad rows again
try:
    pd.to_datetime(bad_rows['datetime-original'])
except Exception as e:
    display(e)

> 🤔"`ParserError: hour must be in 0..23: 10/11/2006 24:00 present at position 0`"

In [ ]:
bad_rows['datetime-original']

> Panda's is confused by 24:00. It doesn't know if we mean 0:00 of the NEXT day or if we mean the 11:59 pm (23:59) the same day

In [ ]:
## replace 24:00 with 23:59
ufo['datetime-fixed'] = ufo["datetime-original"].str.replace(" 24:00"," 23:59")
ufo

In [ ]:
## use the fixed-datetime column to make the datetime col
ufo['datetime'] = pd.to_datetime(ufo["datetime-fixed"])
ufo

In [ ]:
ufo.info()

>- 2) [x] Convert the datetime column  (most likely a string) to a datetime data type.

### Setting datetime index

In [ ]:
## Create ufo_ts by setting the datetime index
ufo_ts = None
ufo_ts

In [ ]:
## Drop any remaining datetime columns
#dt_cols = [c for c in ufo_ts.columns if 'datetime' in c]


In [ ]:
# check the index to confirm its datetime


- 3) [x] Set the datetime column as the Series/DataFrame index

## Let's visualize Our Data

In [ ]:
## Plot the full dataset


> Hmmmm.... what are we *trying* to visualize?



### What do we really want to know about UFO's? 
- Duration of sighting?
- Location of sighting?
- Number of sightings?

## Getting Our Time Series of UFO Sightings

- We want to quantify the number of events that occurred within each interval.

>- Q: How could we do this? (there's several ways)


In [ ]:
## A.1) Add a count column with a value of 1 for each row and then use reasmple().sum()
demo_ts = ufo_ts.copy()


In [ ]:
# A.2) resample and take sum


In [ ]:
## Option B) use .size
ufo_ts.resample("D").size()

### Make `ts` from ufo_ts

In [ ]:
## Resample as daily data 
ts = ufo_ts.resample('D').size()
ts

In [ ]:
# plot the ts


> Let's keep data from 1950 to present day

In [ ]:
## keep only 1950 and later


In [ ]:
# plot again


# Part 2) Aggregating Full Dataset Using Date Parts

## 📝 **Stakeholder Questions to Answer**

**ANSWER TOGETHER:**
- 1) What Month/Year had the most sightings? (and how many sightings were there?)

- 2) Which month of the year has the highest number of reported sightings?
- 3) Is there a seasonal pattern to UFO sightings? If so, how long is the season?

- 4) Which US holiday has the largest number of sightings?
___
**ANSWER SELECTED Q's IN BREAKOUT ROOMS**

- 5) Which year had the highest % increase in sightings compared to previous years? (since 1950)

- 6) What day of the week has the highest reported sightings?

- 7) At what time of day (hour) do most sightings occur?

- 8) Which US state has the most sightings?

- 9) Which country had the largest proportion of sightings for the year 2000?

- 10) Have the types/shapes of UFO's witness changed over time?
    - Tip: use only the 4 most common shapes

### Making `eda_df` for answering questions

In [ ]:
## making eda_df with date as a column instead of index
eda_df = ufo_ts.reset_index()
eda_df

### Feature Engineering: Date Parts

- Datetime objects have:
    - year
    - month
    - month_name()
    - day
    - day_name()
    - hour
    - seconds
    
- Pandas has a `.dt.` accessor to use datetime methods on an entire column at once.

In [ ]:
## feature engineering for dates
eda_df['year'] = eda_df['datetime'].dt.year
eda_df['month'] = eda_df['datetime'].dt.month_name()
eda_df['day of month'] = eda_df['datetime'].dt.day
eda_df['day of week'] = eda_df['datetime'].dt.day_name()
eda_df['hour'] = eda_df['datetime'].dt.hour
eda_df

> Let's add a "weekend" feature that will be True if the day was a Saturday or Sunday.

In [ ]:
## let's add a weekend feature
eda_df['weekend'] = eda_df['day of week'].isin(['Saturday','Sunday'])
eda_df

#### Let's add a column for the decade

In [ ]:
## Calculate decade by subtracting the remainder and div by 10
eda_df['decade'] = eda_df['year'] - eda_df['year']%10
eda_df

## 🕹️ Answering Stakeholder Questions (Together)

### Making `eda_ts` & `ts`

In [ ]:
## Making eda_ts with dt index


### 1) What Month/Year had the most sightings? (and how many sightings were there?)


In [ ]:
## make a ts that is resampled to correct freq


In [ ]:
## get the date of the max sightings
date_most_ufos = None
date_most_ufos

In [ ]:
# how many sightings?


In [ ]:
## Plot the ts and add vertical line at month with most sightings


### 2) Which month of the year has the highest number of reported sightings?


### 3) Is there a seasonal pattern to UFO sightings? If so, how long is the season?

### Seasonality

In [ ]:
import statsmodels.tsa.api as tsa

In [ ]:
ts_check_season = ts_m.loc["2000":]
ts_check_season

In [ ]:
## plot the sliced ts


In [ ]:
## Use seasonal_decompose and plot


In [ ]:
## separate seasonal component and plot


#### Using scipy's find_peaks

In [ ]:
from scipy.signal import find_peaks
peaks, props = find_peaks(seasonal, height=seasonal.max())

In [ ]:
peaks

In [ ]:
peak_dates = seasonal.index[peaks]
peak_dates

### 4) Which US holiday has the largest number of sightings?

#### Feature Engineering: Holidays

In [ ]:
# !pip install holidays
import holidays
import datetime as dt
from holidays import country_holidays

In [ ]:
## Create an instance of the US country holidays.
us_holidays = country_holidays('US')
us_holidays

In [ ]:
## create a test holiday 
test = "01/01/2015"
test

In [ ]:
## test the api 


In [ ]:
## Map the api's .get method onto the df to get all holidays


In [ ]:
## Check the unique holidays

#### Answer to which holiday has most sightings:

#### Wait...when did **that** movie come out?

In [ ]:
release_date= '1997-07-03'

In [ ]:
## Plot the # of sightings over time and annotate the release date


## 🏓**Breakout Rooms: Answering Stakeholder Questions**

**Choose 1-2 of the remaining questions and work in breakout rooms to answer them:**
- 5) Which year had the highest % increase in sightings compared to previous years?
- 6) What day of the week has the highest reported sightings?
- 7) At what time of day (hour) do most sightings occur?
- 8) Which US state has the most sightings?
- 9) Which country had the largest proportion of sightings for the year 2000?
- 10) Have the types/shapes of UFO's witness changed over time?
    - Tip: use only the 4 most common shapes



### 5) Which year had the highest % increase in sightings compared to previous years? (since 1950)

### 6) What day of the week has the highest reported sightings?

In [ ]:
eda_ts['day of week'].value_counts()

### 6) Which country had the largest proportion of sightings for the year 2000?

### 7) Have the types/shapes of UFO's witness changed over time?

___
# Bonus: Plotly Express

In [ ]:
import plotly.express as px
import plotly.io as pio

### Map Over Time

In [ ]:
eda_df = eda_df.sort_values('decade')
eda_df.columns = eda_df.columns.str.strip()
eda_df['latitude'] = pd.to_numeric(eda_df['latitude'], errors='coerce')
eda_df.head()

In [ ]:
px.scatter_geo(data_frame=eda_df, lat='latitude',lon='longitude', animation_frame="decade",
              template='ggplot2')